In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json
import geopandas as gpd
idx = pd.IndexSlice

In [2]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [3]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [4]:
%aimport Data.make_data
from Data.make_data import states_in_nerc
%aimport Analysis.index
from Analysis.index import facility_emission_gen, group_facility_data
%aimport Analysis.index
from Analysis.index import facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport util.utils
from util.utils import rename_cols, add_facility_location

## Load data

In [132]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage',
            'Facility gen fuels and CO2 2017-08-31.zip')
eia_fac = pd.read_csv(path)

In [133]:
rename_cols(eia_fac)

In [7]:
eia_fac.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,M,SUB,6,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-06-01,2,0.0,0.0,0.0,0.0
1,M,SUB,5,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-05-01,2,0.0,0.0,0.0,0.0
2,M,SUB,4,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-04-01,2,0.0,0.0,0.0,0.0
3,M,SUB,3,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-03-01,1,0.0,0.0,0.0,0.0
4,M,SUB,2,10360,0.0,2017,0.0,0.0,USA-WI,2017-08-24T11:46:12-04:00,44.4936,-88.0303,ALL,2017-02-01,1,0.0,0.0,0.0,0.0


In [8]:
path = join(cwd, '..', 'Data storage', 'Derived data',
            'Monthly EPA emissions 2017-08-31.csv')
epa = pd.read_csv(path)

## Test facility_emissions_gen

In [9]:
fuel_cat_folder = join(cwd, '..', 'Data storage', 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

In [113]:
co2, gen_fuels_custom = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=False)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories
Gen/fuels to custom categories


In [10]:
co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=True)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [11]:
co2.head()

,year,month,plant id,final co2 (kg)
0,2001,1,2,0.000000e+00
1,2001,1,3,9.625411e+08
2,2001,1,4,0.000000e+00
3,2001,1,7,6.700092e+07
4,2001,1,8,7.530204e+08


In [11]:
gen_fuels_state.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,COW,2001,1,3,8275496.0,852306.0,8275496.0,31.006900,-88.010300,1,772103776.8,772103776.8,772103776.8,772103776.8
1,COW,2001,1,7,587162.0,45991.0,587162.0,34.012800,-85.970800,1,54782214.6,54782214.6,54782214.6,54782214.6
2,COW,2001,1,8,6910786.0,706857.0,6910786.0,33.644344,-87.196486,1,644776333.8,644776333.8,644776333.8,644776333.8
3,COW,2001,1,10,3108892.0,320454.0,3108892.0,32.601700,-87.781100,1,290059623.6,290059623.6,290059623.6,290059623.6
4,COW,2001,1,26,9745127.0,1005460.0,9745127.0,33.244211,-86.458056,1,909220349.1,909220349.1,909220349.1,909220349.1


### Check generation and fuel consumption totals

Interesting - there is some small part of generation that I'm losing along the way. It's 5 orders of magnitude smaller than the total though.

In [121]:
eia_fac['generation (mwh)'].sum()

65594379743.76258

In [123]:
gen_fuels_state['generation (mwh)'].sum()

65594327524.76352

Fuel consumption is pretty identical though

In [124]:
eia_fac['total fuel (mmbtu)'].sum(), eia_fac['elec fuel (mmbtu)'].sum()

(686393145435.8103, 645653921528.091)

In [125]:
gen_fuels_state['total fuel (mmbtu)'].sum(), gen_fuels_state['elec fuel (mmbtu)'].sum()

(686393145435.7861, 645653921528.0757)

## Extra gen/fuels from non-reporting

In [11]:
%aimport Analysis.index
from Analysis.index import extra_emissions_gen

Total EIA generation/fuel consumption and emission factors

In [12]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage',
            'EIA country-wide gen fuel CO2 2017-08-31.csv')
eia_total = pd.read_csv(path)

path = join(cwd, '..', 'Data storage',
            'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [13]:
eia_total.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
0,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,6,2524250.11,2017-06-01,2,NaN,NaN,0.0,0.0
1,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,5,2479480.14,2017-05-01,2,NaN,NaN,0.0,0.0
2,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,4,2267811.28,2017-04-01,2,NaN,NaN,0.0,0.0
3,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,3,2054274.30,2017-03-01,1,NaN,NaN,0.0,0.0
4,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,2,1449156.85,2017-02-01,1,NaN,NaN,0.0,0.0


In [14]:
extra_co2, extra_gen_fuel = extra_emissions_gen(gen_fuels_state, eia_total, ef)

Results match what I have previously found in the notebooks up on GitHub (Emissions Index repo)

In [119]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16918390.0       1011665.061         13157576.0
          2              14968346.0        911346.865         11574491.0
          3              18321543.0       1153328.913         14765162.0
          4              13530117.0        776341.623         10374008.0
          5              14014024.0        801266.923         10920463.0

In [17]:
extra_gen_fuel.loc[idx['WND',:,:]]

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
WND  2001 1                     NaN      5.566881e+04                NaN
          2                     NaN      1.130211e+05                NaN
          3                     NaN      6.768812e+04                NaN
          4                     NaN      8.794247e+04                NaN
          5                     NaN      8.393153e+04                NaN
          6                     NaN      7.735354e+04                NaN
          7                     NaN      7.101049e+04                NaN
          8                     NaN      5.369600e+04                NaN
          9                     NaN      4.983153e+04                NaN
          10                    NaN      5.820182e+04                NaN
          11                    NaN      2.999030e+04                NaN
          12                    NaN      2.744590e+04                NaN
     2002 1                     NaN      1.614509e+03                NaN
          2                     NaN      1.495079e+03                NaN
          3                     NaN      1.805048e+03                NaN
          4                     NaN      1.512223e+03                NaN
          5                     NaN      1.524876e+03                NaN
          6                     NaN      1.855552e+03                NaN
          7                     NaN      2.156177e+03                NaN
          8                     NaN      2.061015e+03                NaN
          9                     NaN      1.887244e+03                NaN
          10                    NaN      1.644759e+03                NaN
          11                    NaN      1.521470e+03                NaN
          12                    NaN      1.722605e+03                NaN
     2003 1                     NaN      4.380000e-01                NaN
          2                     NaN     -2.490000e-01                NaN
          3                     NaN     -2.510000e-01                NaN
          4                     NaN     -2.110000e-01                NaN
          5                     NaN     -2.610000e-01                NaN
          6                     NaN     -3.660000e-01                NaN
...                             ...               ...                ...
     2015 1                     NaN      3.183256e+03                NaN
          2                     NaN      3.654121e+03                NaN
          3                     NaN      3.460416e+03                NaN
          4                     NaN      4.872189e+03                NaN
          5                     NaN      5.320185e+03                NaN
          6                     NaN      4.493551e+03                NaN
          7                     NaN      4.591517e+03                NaN
          8                     NaN      4.083333e+03                NaN
          9                     NaN      3.027033e+04                NaN
          10                    NaN      3.567217e+03                NaN
          11                    NaN      4.449315e+03                NaN
          12                    NaN      5.930371e+03                NaN
     2016 1                     NaN      6.453995e+05                NaN
          2                     NaN      6.895701e+05                NaN
          3                     NaN      7.570480e+05                NaN
          4                     NaN      7.213771e+05                NaN
          5                     NaN      6.494653e+05                NaN
          6                     NaN      5.751191e+05                NaN
          7                     NaN      5.831914e+05                NaN
          8                     NaN      4.499372e+05                NaN
          9                     NaN      5.419517e+05                NaN
          10                    NaN      6.582283e+05                NaN
  

## Total CO₂ (national)

In [15]:
facility_co2 = co2.groupby(['year', 'month']).sum()

In [16]:
facility_co2.tail()

plant id  final co2 (kg)
year month                          
2017 2      57482310    1.165442e+11
     3      58485598    1.237187e+11
     4      58955690    1.133454e+11
     5      59223601    1.292581e+11
     6      59451819    1.512113e+11

In [17]:
extra_co2.loc[idx['NG', :, :],:].tail(n=11)

all fuel co2 (kg)  elec fuel co2 (kg)
type year month                                       
NG   2016 8           6.022574e+09        4.039321e+09
          9           4.917984e+09        3.118869e+09
          10          4.312456e+09        2.624005e+09
          11          4.231444e+09        2.458983e+09
          12          4.418713e+09        2.530048e+09
     2017 1           7.823823e+09        6.101011e+09
          2           6.601814e+09        5.030973e+09
          3           8.630216e+09        6.986014e+09
          4           7.823192e+09        6.280070e+09
          5           8.749634e+09        7.244919e+09
          6           1.000377e+10        8.465898e+09

In [18]:
national_co2 = (facility_co2.loc[:, 'final co2 (kg)']
                + extra_co2.loc[:, 'elec fuel co2 (kg)'].groupby(['year', 'month']).sum())

These values are very close, but not quite exactly the same as what I've put up on the website.

Why? Could it be because the extra 

In [126]:
national_co2

year  month
2001  1        2.162121e+11
      2        1.814475e+11
      3        1.900841e+11
      4        1.761317e+11
      5        1.905779e+11
      6        2.062662e+11
      7        2.311815e+11
      8        2.404311e+11
      9        1.950060e+11
      10       1.845210e+11
      11       1.731834e+11
      12       1.882633e+11
2002  1        1.951862e+11
      2        1.709727e+11
      3        1.855136e+11
      4        1.759043e+11
      5        1.863580e+11
      6        2.081023e+11
      7        2.385440e+11
      8        2.355222e+11
      9        2.092338e+11
      10       1.948277e+11
      11       1.844629e+11
      12       2.023788e+11
2003  1        2.167834e+11
      2        1.880347e+11
      3        1.876779e+11
      4        1.727763e+11
      5        1.840170e+11
      6        2.007602e+11
                   ...     
2015  1        1.779027e+11
      2        1.696220e+11
      3        1.522496e+11
      4        1.310557e+11
      5 

## National Index and gen by fuels

In [127]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16918390.0       1011665.061         13157576.0
          2              14968346.0        911346.865         11574491.0
          3              18321543.0       1153328.913         14765162.0
          4              13530117.0        776341.623         10374008.0
          5              14014024.0        801266.923         10920463.0

In [136]:
gen_fuels_state.groupby(['year', 'month']).sum().tail()

plant id  total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)  \
year month                                                                      
2017 2      89637881        2.471045e+09      2.462247e+08       2.349592e+09   
     3      90951249        2.659571e+09      2.654742e+08       2.526846e+09   
     4      91388289        2.424457e+09      2.422983e+08       2.304171e+09   
     5      91583293        2.676784e+09      2.681190e+08       2.555053e+09   
     6      91677954        2.997427e+09      3.021853e+08       2.876122e+09   

                      lat            lon  quarter  all fuel fossil co2 (kg)  \
year month                                                                    
2017 2      120729.133612 -296504.534805     3192              1.209563e+11   
     3      121757.578709 -298889.971201     3217              1.285636e+11   
     4      121805.826865 -299282.190963     6440              1.176367e+11   
     5      121961.825569 -299520.312134     6446              1.334064e+11   
     6      121896.867105 -299481.456675     6444              1.544630e+11   

            elec fuel fossil co2 (kg)  all fuel total co2 (kg)  \
year month                                                       
2017 2                   1.155701e+11             1.273938e+11   
     3                   1.225800e+11             1.354570e+11   
     4                   1.122119e+11             1.239456e+11   
     5                   1.278850e+11             1.397844e+11   
     6                   1.490479e+11             1.609368e+11   

            elec fuel total co2 (kg)  
year month                            
2017 2                  1.180525e+11  
     3                  1.252646e+11  
     4                  1.147064e+11  
     5                  1.304835e+11  
     6                  1.516790e+11

In [19]:
national_gen = (gen_fuels_state
                .groupby(['type', 'year', 'month'])['generation (mwh)'].sum()
                .add(extra_gen_fuel['generation (mwh)'], fill_value=0))

In [20]:
national_gen.groupby(['year', 'month']).sum().tail()

year  month
2017  2        2.898631e+08
      3        3.199879e+08
      4        2.959468e+08
      5        3.236810e+08
      6        3.582983e+08
Name: generation (mwh), dtype: float64

In [21]:
national_gen = group_fuel_cats(national_gen.reset_index(), custom_fuel_cat,
                               'type', 'fuel category').set_index(['fuel category', 'year', 'month'])

In [22]:
national_gen.tail()

generation (mwh)
fuel category year month                  
Wind          2017 2           21691634.88
                   3           25598907.50
                   4           25403089.66
                   5           22326395.36
                   6           19428938.56

In [23]:
total_gen = national_gen.groupby(['year', 'month']).sum()
# percent_gen = national_gen / total_gen
total_gen.head()

generation (mwh)
year month                  
2001 1           332493160.0
     2           282940198.0
     3           300706544.0
     4           278078871.0
     5           300491621.0

In [24]:
df_list = []
for fuel in national_gen.index.get_level_values('fuel category').unique():
    percent_gen = national_gen.loc[fuel].divide(total_gen, fill_value=0)
    percent_gen['fuel category'] = fuel
    percent_gen.set_index('fuel category', inplace=True, append=True)
    df_list.append(percent_gen)
percent_gen = pd.concat(df_list)

In [162]:
path = join(cwd, '..', 'Data storage', 'National data', 'National generation.csv')
national_gen.to_csv(path)

path = join(cwd, '..', 'Data storage', 'National data', 'National percent gen.csv')
percent_gen.to_csv(path)

## Facility state and lat/lon file generation

In [14]:
%aimport Data.make_data
from Data.make_data import facility_location_data

In [15]:
facility_location_data(eia_fac)

## Fraction of estimated gen/fuels in each NERC region

**NOTE** EIA changed the reporting requirements for wind and solar facilities in 2017. I'm now going to use the list of annual facilities from 2017 rather than from 2015, but still use the reported generation by those facilities in 2015.

In [25]:
%aimport Data.make_data
from Data.make_data import get_annual_plants

In [34]:
annual_ids_2015 = get_annual_plants(2015)

In [52]:
annual_ids_2016 = get_annual_plants(2016)

In [29]:
annual_ids_2017 = get_annual_plants(2017)

Combine the lists of annual plants in 2015 and 2017. This lets us catch facilities that have gone from monthly to annual since 2015, but it also includes plants that were annual in 2015 an may have retired.

There is the possibility of an error in allocation to NERC regions for 2016 state-level generation when using the facilities that changed to annual in 2017. But since the state-level generation is so much smaller in 2016 I don't think this is much of an issue.

In [160]:
annual_ids = set(annual_ids_2015.tolist() + annual_ids_2017.tolist())

In [161]:
len(annual_ids)

7146

### Add NERC region labels

In [59]:
%aimport Analysis.state2nerc
from Analysis.state2nerc import fraction_state2nerc, add_region

In [72]:
cwd = os.getcwd()
path = join(cwd, '..', 'Data storage', 'Facility labels',
            'Facility locations.csv')
location_labels = pd.read_csv(path)

In [73]:
nerc_state_path = join(cwd, '..', 'Data storage', 'Derived data',
                  'NERC_states.json')

with open(nerc_state_path, 'r') as f:
    nerc_states = json.load(f)

Added the filter that year must be 2015 - was getting all 2015 annual plants, but for all years!

In [162]:
eia_2015_annual = eia_fac.loc[(eia_fac['plant id'].isin(annual_ids)) & 
                              (eia_fac['year'] == 2015)].copy()

# Group to state-level fuel categories
eia_2015_annual = group_fuel_cats(eia_2015_annual, state_fuel_cat)

In [165]:
eia_2015_annual_nerc = add_facility_location(eia_2015_annual, location_labels, 
                                        labels=['state', 'nerc'])

In [167]:
eia_2015_annual_nerc.tail()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),state,nerc
89678,WWW,2015,8,58707,211987.0,13966.0,211987.0,33.238611,-80.450278,3,0.0,0.0,19884380.6,19884380.6,SC,SERC
89679,WWW,2015,9,58707,214505.0,13769.0,214505.0,33.238611,-80.450278,3,0.0,0.0,20120569.0,20120569.0,SC,SERC
89680,WWW,2015,10,58707,162426.0,8554.0,162426.0,33.238611,-80.450278,4,0.0,0.0,15235558.8,15235558.8,SC,SERC
89681,WWW,2015,11,58707,245845.0,13352.0,245845.0,33.238611,-80.450278,4,0.0,0.0,23060261.0,23060261.0,SC,SERC
89682,WWW,2015,12,58707,249553.0,14099.0,249553.0,33.238611,-80.450278,4,0.0,0.0,23408071.4,23408071.4,SC,SERC


In [122]:
# Get a list of all state abbreviations

all_states = []
for value in nerc_states.values():
    all_states.extend(value)
all_states = set(all_states)

In [169]:
df_list = []

for state in all_states:
    try:
        df_list.append(fraction_state2nerc(eia_2015_annual_nerc,
                       state, region_col='nerc', fuel_col='type'))
    except:
        print(state)
        pass

DC


In [170]:
nerc_fraction = pd.concat(df_list)
nerc_fraction.set_index(['state', 'nerc', 'type'], inplace=True)
nerc_fraction.sort_index(inplace=True)

In [171]:
nerc_fraction.loc[idx['TX', :, 'WND'], :]

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    SPP  WND       0.213313      0.213313     0.213313
      TRE  WND       0.786687      0.786687     0.786687
      WECC WND       0.000000      0.000000     0.000000

With the values below I can allocate extra state-level generation and fuel use to each of the NERC regions!

In [172]:
nerc_fraction.loc['TX']

% generation  % total fuel  % elec fuel
nerc type                                         
SPP  NG        0.155345      0.159005     0.181090
     OOG       0.282734      0.159436     0.284516
     PEL       0.019479      0.005109     0.018611
     WAS       0.892327      0.958075     0.847920
     WND       0.213313      0.213313     0.213313
     WWW       0.609573      0.282686     0.398931
TRE  HYC       1.000000      1.000000     1.000000
     NG        0.782840      0.784903     0.747629
     OOG       0.717266      0.840564     0.715484
     OTH       1.000000      1.000000     1.000000
     PC        1.000000      1.000000     1.000000
     PEL       0.900883      0.978261     0.920816
     SUN       0.932089      0.932089     0.932089
     WAS       0.107673      0.041925     0.152080
     WND       0.786687      0.786687     0.786687
     WWW       0.390427      0.717314     0.601069
WECC NG        0.061815      0.056091     0.071282
     PEL       0.079638      0.016630     0.060573
     SUN       0.067911      0.067911     0.067911
     WND       0.000000      0.000000     0.000000

In [173]:
nerc_fraction.tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
WY    WECC NG             1.0           1.0          1.0
           OOG            1.0           1.0          1.0
           OTH            1.0           1.0          1.0
           PEL            1.0           1.0          1.0
           WND            1.0           1.0          1.0

In [174]:
nerc_fraction.groupby(['state', 'type']).sum()

% generation  % total fuel  % elec fuel
state type                                         
AL    COW            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            NaN           NaN          NaN
      PEL            1.0           1.0          1.0
      WAS            NaN           NaN          NaN
      WWW            1.0           1.0          1.0
AR    HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PC             NaN           NaN          NaN
      PEL            1.0           1.0          1.0
      SUN            1.0           1.0          1.0
      WAS            1.0           1.0          1.0
      WWW            1.0           1.0          1.0
AZ    HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      SUN            1.0           1.0          1.0
      WND            1.0           1.0          1.0
      WWW            1.0           1.0          1.0
CA    COW            NaN           NaN          NaN
      GEO            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
...                  ...           ...          ...
WA    HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      WAS            1.0           1.0          1.0
      WND            1.0           1.0          1.0
      WWW            1.0           1.0          1.0
WI    COW            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PC             NaN           NaN          NaN
      PEL            1.0           1.0          1.0
      SUN            1.0           1.0          1.0
      WAS            1.0           1.0          1.0
      WND            1.0           1.0          1.0
      WWW            1.0           1.0          1.0
WV    COW            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      WND            1.0           1.0          1.0
WY    COW            1.0           1.0          1.0
      HYC            1.0           1.0          1.0
      NG             1.0           1.0          1.0
      OOG            1.0           1.0          1.0
      OTH            1.0           1.0          1.0
      PEL            1.0           1.0          1.0
      WND            1.0           1.0          1.0

[418 rows x 3 columns]

## Allocate extra gen from the state-level to regions

I still need to generate state-level total generation and fuel use!

In [80]:
idx = pd.IndexSlice

In [175]:
# a dictionary to match column names
nerc_frac_match = {'% generation': 'generation (mwh)',
                   '% total fuel': 'total fuel (mmbtu)',
                   '% elec fuel': 'elec fuel (mmbtu)'}

In [176]:
path = join(cwd, '..', 'Data storage', 'EIA state-level gen fuel CO2.csv')

state_total = pd.read_csv(path, parse_dates=['datetime'])

In [177]:
rename_cols(state_total)

In [178]:
state_total.head()

,end,f,last_updated,sector,series_id,start,units,generation (mwh),total fuel (mmbtu),elec fuel (mmbtu),all fuel co2 (kg),elec fuel co2 (kg),datetime,quarter
0,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AK-99.M,200101.0,megawatthours,87.00,NaN,NaN,NaN,NaN,2001-01-01,1
1,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AL-99.M,200101.0,megawatthours,401167.59,NaN,NaN,NaN,NaN,2001-01-01,1
2,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AR-99.M,200101.0,megawatthours,136530.37,NaN,NaN,NaN,NaN,2001-01-01,1
3,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AZ-99.M,200101.0,megawatthours,453.00,NaN,NaN,NaN,NaN,2001-01-01,1
4,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-CA-99.M,200101.0,megawatthours,1717398.41,NaN,NaN,NaN,NaN,2001-01-01,1


In [179]:
state_total.dtypes

end                          float64
f                             object
last_updated                  object
sector                       float64
series_id                     object
start                        float64
units                         object
generation (mwh)             float64
total fuel (mmbtu)           float64
elec fuel (mmbtu)            float64
all fuel co2 (kg)            float64
elec fuel co2 (kg)           float64
datetime              datetime64[ns]
quarter                        int64
dtype: object

In [180]:
state_total['state'] = state_total['series_id'].str.split('-').str[1]
state_total['type'] = (state_total['series_id'].str.split('-').str[0].str.split('.').str[-1])
state_total['year'] = state_total['datetime'].dt.year
state_total['month'] = state_total['datetime'].dt.month

In [181]:
state_total.head()

,end,f,last_updated,sector,series_id,start,units,generation (mwh),total fuel (mmbtu),elec fuel (mmbtu),all fuel co2 (kg),elec fuel co2 (kg),datetime,quarter,state,type,year,month
0,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AK-99.M,200101.0,megawatthours,87.00,NaN,NaN,NaN,NaN,2001-01-01,1,AK,AOR,2001,1
1,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AL-99.M,200101.0,megawatthours,401167.59,NaN,NaN,NaN,NaN,2001-01-01,1,AL,AOR,2001,1
2,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AR-99.M,200101.0,megawatthours,136530.37,NaN,NaN,NaN,NaN,2001-01-01,1,AR,AOR,2001,1
3,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AZ-99.M,200101.0,megawatthours,453.00,NaN,NaN,NaN,NaN,2001-01-01,1,AZ,AOR,2001,1
4,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-CA-99.M,200101.0,megawatthours,1717398.41,NaN,NaN,NaN,NaN,2001-01-01,1,CA,AOR,2001,1


In [182]:
cols = list(nerc_frac_match.values())
state_total = state_total.groupby(['state', 'year', 'month', 'type'])[cols].sum()
state_total.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     AOR               87.0                 NaN                NaN
                 COW            46903.0           1120000.0           872000.0
                 HYC           104549.0                 NaN                NaN
                 NG            367521.0           4091000.0          3989000.0
                 PEL            71085.0            767000.0           763000.0

In [183]:
state_total.loc[idx['KS', 2017, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
KS    2017 1     WND         1418930.40                 NaN                NaN
           2     WND         1428272.57                 NaN                NaN
           3     WND         1879884.72                 NaN                NaN
           4     WND         1922784.64                 NaN                NaN
           5     WND         1638581.51                 NaN                NaN
           6     WND         1463920.68                 NaN                NaN

In [184]:
state_total.loc[idx['OK', 2017, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
OK    2017 1     WND         2042596.53                 NaN                NaN
           2     WND         2123281.94                 NaN                NaN
           3     WND         2497964.62                 NaN                NaN
           4     WND         2408824.80                 NaN                NaN
           5     WND         2021006.60                 NaN                NaN
           6     WND         1879154.53                 NaN                NaN

In [185]:
nercs = nerc_fraction.index.get_level_values('nerc').unique()

In [186]:
cols = list(nerc_frac_match.values())
eia_fac_type = group_fuel_cats(eia_fac, state_fuel_cat)
eia_fac_type = add_facility_location(eia_fac_type, location_labels, ['state'])
eia_fac_type = eia_fac_type.groupby(['state', 'year', 'month', 'type'])[cols].sum()

In [187]:
eia_fac_type.loc[idx['OK', 2017, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
OK    2017 1     WND           982955.0           9160159.0          9160159.0
           2     WND          1053595.0           9818451.0          9818451.0
           3     WND          1237665.0          11533802.0         11533802.0
           4     WND          1164423.0          10851260.0         10851260.0
           5     WND           960103.0           8947200.0          8947200.0
           6     WND           987307.0           9200713.0          9200713.0

In [188]:
eia_fac_type.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AL    2001 1     COW        6547312.020         67581702.00        66429143.00
                 HYC         767601.000          7931621.15         7931621.15
                 NG          503553.867          7213851.00         4689532.00
                 NUC        2940300.000         30702612.00        30702612.00
                 OOG           2828.126           164765.00           31285.00

In [189]:
(state_total.loc[idx[:, 2016:, :, :], :]
 - eia_fac_type.loc[idx[:, 2016:, :, :], :]).loc[idx['OK', 2017, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
OK    2017 1     WND         1059641.53                 NaN                NaN
           2     WND         1069686.94                 NaN                NaN
           3     WND         1260299.62                 NaN                NaN
           4     WND         1244401.80                 NaN                NaN
           5     WND         1060903.60                 NaN                NaN
           6     WND          891847.53                 NaN                NaN

In [190]:
state_extra = (state_total.loc[idx[:, 2016:, :, :], :]
               - eia_fac_type.loc[idx[:, 2016:, :, :], :])
state_extra.dropna(how='all', inplace=True)
state_extra = state_extra.reorder_levels(['year', 'state', 'month', 'type'])
state_extra.sort_index(inplace=True)
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AL    1     COW           3172.785            143129.0            28261.0
                 HYC         300276.300                 NaN                NaN
                 NG          149094.522           2875066.0          1677549.0
                 NUC              0.000                 NaN                NaN
                 PEL           1987.279             89981.0            17614.0

The huge jump in Texas state-level wind generation is why I need to use more recent lists of annual reporting facilities rather than the list from 2015.

In [191]:
state_extra.loc[idx[:, 'TX', :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 TX    1     WND           90562.51                 NaN                NaN
           2     WND          102900.59                 NaN                NaN
           3     WND          123610.89                 NaN                NaN
           4     WND          107811.34                 NaN                NaN
           5     WND          112386.52                 NaN                NaN
           6     WND           91352.99                 NaN                NaN
           7     WND          115339.50                 NaN                NaN
           8     WND           82661.79                 NaN                NaN
           9     WND           85334.20                 NaN                NaN
           10    WND          108237.19                 NaN                NaN
           11    WND           87830.32                 NaN                NaN
           12    WND           96075.55                 NaN                NaN
2017 TX    1     WND         2991690.73                 NaN                NaN
           2     WND         2924136.52                 NaN                NaN
           3     WND         3451667.09                 NaN                NaN
           4     WND         3422068.33                 NaN                NaN
           5     WND         3037395.57                 NaN                NaN
           6     WND         2438002.79                 NaN                NaN

In [192]:
state_extra.loc[2016].head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state month type                                                         
AL    1     COW           3172.785            143129.0            28261.0
            HYC         300276.300                 NaN                NaN
            NG          149094.522           2875066.0          1677549.0
            NUC              0.000                 NaN                NaN
            PEL           1987.279             89981.0            17614.0

In [193]:
state_extra.loc[(state_extra < -100).any(axis=1)]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AZ    6     HYC         -12722.670                 NaN                NaN
           7     HYC         -12463.360                 NaN                NaN
           8     HYC          -6672.920                 NaN                NaN
     CA    1     OOG         -36609.329                 NaN                NaN
           2     OOG         -35875.716                 NaN                NaN
           3     OOG         -35981.849                 NaN                NaN
           4     OOG         -25453.429                 NaN                NaN
           5     OOG         -37564.717                 NaN                NaN
           6     OOG         -36266.771                 NaN                NaN
           7     OOG         -34884.006                 NaN                NaN
           8     OOG         -35066.275                 NaN                NaN
           9     OOG         -37674.828                 NaN                NaN
           10    OOG         -36968.936                 NaN                NaN
           11    OOG         -33645.069                 NaN                NaN
           12    OOG         -36549.401                 NaN                NaN
     FL    1     OOG          -2215.246                 NaN                NaN
           2     OOG          -6301.215                 NaN                NaN
           3     OOG          -7743.011                 NaN                NaN
           4     OOG          -7163.809                 NaN                NaN
           5     OOG          -7527.673                 NaN                NaN
           6     OOG          -4412.662                 NaN                NaN
           7     OOG          -3398.979                 NaN                NaN
           8     OOG          -5283.892                 NaN                NaN
           9     OOG          -5375.453                 NaN                NaN
           10    OOG          -5124.908                 NaN                NaN
           11    OOG          -5939.829                 NaN                NaN
           12    OOG          -5858.501                 NaN                NaN
     IN    1     PEL        -169080.314          -1227573.0         -1227775.0
           2     PEL        -150481.598          -1088481.0         -1088690.0
           3     PEL         -40284.979           -350888.0          -351000.0
...                                 ...                 ...                ...
2017 MI    5     OOG         -27008.003                 NaN                NaN
           6     OOG         -23453.997                 NaN                NaN
     NJ    1     OOG         -19288.000                 NaN                NaN
           2     OOG         -17994.000                 NaN                NaN
           3     OOG         -16372.000                 NaN                NaN
           4     OOG         -14897.000                 NaN                NaN
           5     OOG         -11254.000                 NaN                NaN
           6     OOG         -17793.000                 NaN                NaN
     NY    1     OOG         -11203.000                 NaN                NaN
           2     OOG         -10535.000                 NaN                NaN
           3     OOG         -11829.000                 NaN                NaN
           4     OOG         -11114.000                 NaN                NaN
           5     OOG         -10736.000                 NaN                NaN
           6     OOG          -9741.000                 NaN                NaN
     OH    5     OTH           -829.004                 NaN                NaN
           6     OTH           -808.997                 NaN                NaN
     PA    1     OOG          -5416.866                 NaN                NaN
           4     OOG          -1904.495                 NaN              

Why is generation from facilities greater than state-level estimates in some cases?

In [195]:
nerc_fraction.sort_index(inplace=True)
nerc_fraction.loc[idx['TX', 'WECC', :], :].tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    WECC NG        0.061815      0.056091     0.071282
           PEL       0.079638      0.016630     0.060573
           SUN       0.067911      0.067911     0.067911
           WND       0.000000      0.000000     0.000000

In [196]:
state_extra.sort_index(inplace=True)
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AL    1     COW           3172.785            143129.0            28261.0
                 HYC         300276.300                 NaN                NaN
                 NG          149094.522           2875066.0          1677549.0
                 NUC              0.000                 NaN                NaN
                 PEL           1987.279             89981.0            17614.0

In [197]:
state_extra.loc[2016, :].head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state month type                                                         
AL    1     COW           3172.785            143129.0            28261.0
            HYC         300276.300                 NaN                NaN
            NG          149094.522           2875066.0          1677549.0
            NUC              0.000                 NaN                NaN
            PEL           1987.279             89981.0            17614.0

In [96]:
state_extra_2016 = state_extra.loc[2016, :]
# state_extra_2016.index = state_extra_2016.index.droplevel('year')

state_extra_2017 = state_extra.loc[2017, :]
# state_extra_2017.index = state_extra_2017.index.droplevel('year')

In [97]:
state_extra_2016.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state month type                                                         
AL    1     COW           3172.785            143129.0            28261.0
            HYC         300276.300                 NaN                NaN
            NG          149094.522           2875066.0          1677549.0
            NUC              0.000                 NaN                NaN
            PEL           1987.279             89981.0            17614.0

In [198]:
df_list = []
for month in range(1, 13):
    df = nerc_fraction.copy()
    df['month'] = month
    df.set_index('month', append=True, inplace=True)
    df_list.append(df)

nerc_frac_monthly = pd.concat(df_list, axis=0)
nerc_frac_monthly.sort_index(inplace=True)
nerc_frac_monthly = (nerc_frac_monthly
                     .reorder_levels(['nerc', 'state', 'month', 'type']))

This is it!

In [199]:
nerc_frac_monthly.loc['TRE'].head()

% generation  % total fuel  % elec fuel
state month type                                         
NM    1     SUN         0.0693        0.0693       0.0693
      2     SUN         0.0693        0.0693       0.0693
      3     SUN         0.0693        0.0693       0.0693
      4     SUN         0.0693        0.0693       0.0693
      5     SUN         0.0693        0.0693       0.0693

In [63]:
nercs

Index(['SERC', '-', 'SPP', 'WECC', 'NPCC', 'RFC', 'FRCC', 'MRO', 'TRE'], dtype='object', name='nerc')

In [200]:
df_list_outer = []
for year in [2016, 2017]:
    df_list_inner = []
    for nerc in nercs:
        df = pd.concat([(nerc_frac_monthly
                         .loc[nerc]['% generation']
                         * state_extra
                         .loc[year]['generation (mwh)']).dropna(),
                        (nerc_frac_monthly.
                         loc[nerc]['% total fuel']
                         * state_extra
                         .loc[year]['total fuel (mmbtu)']).dropna(),
                        (nerc_frac_monthly
                         .loc[nerc]['% elec fuel']
                         * state_extra
                         .loc[year]['elec fuel (mmbtu)']).dropna()],
                        axis=1)
        df.columns = nerc_frac_match.values()
        df['nerc'] = nerc
        df['year'] = year
        df = df.groupby(['year', 'nerc', 'month', 'type']).sum()
        df_list_inner.append(df)

    df_list_outer.append(pd.concat(df_list_inner))
final = pd.concat(df_list_outer)
final.sort_index(inplace=True)

In [201]:
df_list_inner[2].loc[idx[:, :, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2017 SPP  1     WND       2.373849e+06                 NaN                NaN
          2     WND       2.379149e+06                 NaN                NaN
          3     WND       2.799971e+06                 NaN                NaN
          4     WND       2.764657e+06                 NaN                NaN
          5     WND       2.389331e+06                 NaN                NaN
          6     WND       1.982435e+06                 NaN                NaN

In [121]:
df_list_inner[2].loc[idx[:, :, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2017 SPP  1     WND       3.413155e+06                 NaN                NaN
          2     WND       3.392016e+06                 NaN                NaN
          3     WND       3.998834e+06                 NaN                NaN
          4     WND       3.956044e+06                 NaN                NaN
          5     WND       3.444754e+06                 NaN                NaN
          6     WND       2.830841e+06                 NaN                NaN

In [202]:
nerc_frac_monthly.sort_index(inplace=True)

In [205]:
((state_extra.loc[2017].loc['TX']['generation (mwh)']
 * nerc_frac_monthly.loc['SPP'].loc['TX']['% generation'])
 .groupby(['month', 'type']).sum()
 .loc[idx[:, 'WND']])

month
1     638166.805470
2     623756.607932
3     736285.785922
4     729971.982852
5     647916.246295
6     520057.917959
7               NaN
8               NaN
9               NaN
10              NaN
11              NaN
12              NaN
dtype: float64

In [122]:
((state_extra.loc[2017].loc['TX']['generation (mwh)']
 * nerc_frac_monthly.loc['SPP'].loc['TX']['% generation'])
 .groupby(['month', 'type']).sum()
 .loc[idx[:, 'WND']])

month
1     1.617639e+06
2     1.581112e+06
3     1.866353e+06
4     1.850349e+06
5     1.642352e+06
6     1.318254e+06
7              NaN
8              NaN
9              NaN
10             NaN
11             NaN
12             NaN
dtype: float64

In [206]:
final.loc[idx[2017, 'SPP', :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2017 SPP  1     WND       2.373849e+06                 NaN                NaN
          2     WND       2.379149e+06                 NaN                NaN
          3     WND       2.799971e+06                 NaN                NaN
          4     WND       2.764657e+06                 NaN                NaN
          5     WND       2.389331e+06                 NaN                NaN
          6     WND       1.982435e+06                 NaN                NaN

In [105]:
final

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2016 SERC 1     COW       1.250252e+05        4.777255e+06       1.075619e+06
                HYC       1.578592e+06                 NaN                NaN
                NG        6.487438e+05        1.456982e+07       5.659672e+06
                OOG       2.896880e+04                 NaN                NaN
                OTH       1.081877e+05                 NaN                NaN
                PC        9.706641e+03        3.439330e+05       4.924300e+04
                PEL       1.467958e+04        2.632000e+05       1.213191e+05
                SUN       1.530236e+05                 NaN                NaN
                WAS       1.460550e+05                 NaN                NaN
                WND       2.933270e+03                 NaN                NaN
                WWW       5.505123e+05                 NaN                NaN
          2     COW       1.096822e+05        4.247556e+06       8.969419e+05
                HYC       1.422324e+06                 NaN                NaN
                NG        5.949623e+05        1.374200e+07       5.163385e+06
                OOG       2.626671e+04                 NaN                NaN
                OTH       9.516741e+04                 NaN                NaN
                PC        1.428786e+04        4.354360e+05       7.247600e+04
                PEL       8.375256e+03        1.463122e+05       6.522541e+04
                SUN       2.686865e+05                 NaN                NaN
                WAS       1.280215e+05                 NaN                NaN
                WND       3.305718e+03                 NaN                NaN
                WWW       5.137783e+05                 NaN                NaN
          3     COW       9.485107e+04        3.890549e+06       7.402895e+05
                HYC       1.165033e+06                 NaN                NaN
                NG        6.751874e+05        1.450502e+07       5.884467e+06
                OOG       2.883483e+04                 NaN                NaN
                OTH       1.067343e+05                 NaN                NaN
                PC        1.851254e+04        5.076600e+05       9.391000e+04
                PEL       6.407982e+03        8.983574e+04       4.747977e+04
                SUN       2.930856e+05                 NaN                NaN
...                                ...                 ...                ...
2017 TRE  4     HYC       1.032891e+05                 NaN                NaN
                NG        3.361071e+06        3.723031e+07       2.851581e+07
                OOG       3.592197e+04                 NaN                NaN
                OTH       3.620240e+03                 NaN                NaN
                PC        0.000000e+00        1.006100e+04      -4.000000e+00
                PEL       2.303771e+01        2.171757e+02       3.065437e+02
                SUN       9.248117e+04                 NaN                NaN
                WAS       1.533996e+04                 NaN                NaN
                WND       6.339001e+05                 NaN                NaN
                WWW       2.542933e+04                 NaN                NaN
          5     HYC       1.264384e+05                 NaN                NaN
                NG        3.600007e+06        3.985479e+07       3.140188e+07
                OOG       3.692411e+04                 NaN                NaN
                OTH       4.674410e+03                 NaN                NaN
                PC        4.283650e+02        1.966200e+04       5.025000e+03
                PEL       2.383657e+01        2.141016e+02       3.036934e+02
                SUN       1.142562e+05                 NaN                NaN
                WAS       1.606144e+04                 NaN                NaN
                WND       5.626438e+05      

In [207]:
path = join(cwd, '..', 'Data storage', 'NERC extra gen fuels_rev.csv')
final.to_csv(path)

## Check total generation in NPCC
Easy state boundaries

In [107]:
npcc_state = ['NY', 'CT', 'RI', 'MA', 'VT', 'NH', 'ME']

In [108]:
state_total.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     AOR               87.0                 NaN                NaN
                 COW            46903.0           1120000.0           872000.0
                 HYC           104549.0                 NaN                NaN
                 NG            367521.0           4091000.0          3989000.0
                 PEL            71085.0            767000.0           763000.0

In [ ]:
types = state

In [112]:
state_total.loc[idx[npcc_state, :, :, :], :].groupby(['year', 'month']).sum()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year month                                                         
2001 1           23460172.75         142758000.0        132585000.0
     2           20862523.25         118389000.0        108740000.0
     3           22436317.43         130155000.0        119528000.0
     4           19788902.63         100303000.0         91749000.0
     5           20521924.71         114841000.0        106227000.0
     6           23884138.30         133834000.0        125326000.0
     7           24133560.44         140101000.0        131259000.0
     8           27082507.52         172179000.0        162340000.0
     9           22354372.48         125217000.0        116323000.0
     10          22024272.79         121355000.0        112875000.0
     11          20701829.50         106996000.0         98193000.0
     12          22615347.10         122403000.0        112810000.0
2002 1           23092512.27         121551000.0        111172000.0
     2           20108926.99         102537000.0         94455000.0
     3           21568699.02         112828000.0        103720000.0
     4           20032579.35         100951000.0         92222000.0
     5           21371510.10         107295000.0         99461000.0
     6           23253412.44         122936000.0        114922000.0
     7           26618200.27         162446000.0        153676000.0
     8           26430581.51         162456000.0        153958000.0
     9           22958695.73         133771000.0        125292000.0
     10          22097137.46         125367000.0        116944000.0
     11          21806135.80         122374000.0        113520000.0
     12          23866772.60         131496000.0        121654000.0
2003 1           24567588.33         134751000.0        126325000.0
     2           21877246.76         123279000.0        115949000.0
     3           22243480.20         122156000.0        115014000.0
     4           20776107.10         114687000.0        108150000.0
     5           20429435.40          93658000.0         88057000.0
     6           22333250.11         117058000.0        111036000.0
...                      ...                 ...                ...
2015 1           23246942.58          90008780.0         84063810.0
     2           22232818.70         102254450.0         95308300.0
     3           21852880.08          84522170.0         78397280.0
     4           19889187.00          63526970.0         58918900.0
     5           21873033.68          81802470.0         78525790.0
     6           22478705.05          83179240.0         79660270.0
     7           26277680.38         115238040.0        111089540.0
     8           26223071.47         116006030.0        111471930.0
     9           22956484.87          97185790.0         92520470.0
     10          21205996.42          84893070.0         80621740.0
     11          20505985.34          70843410.0         66822090.0
     12          21422708.43          68124770.0         63941060.0
2016 1           22540205.51          78362350.0         72867720.0
     2           21241529.52          72324710.0         66813780.0
     3           20788397.41          66866660.0         62044050.0
     4           19491853.37          72868120.0         69106050.0
     5           21654504.06          83693140.0         80217800.0
     6           22446127.97          91713610.0         87928100.0
     7           26642347.24         122177910.0        116936360.0
     8           27198423.81         129362580.0        123664620.0
     9           22255865.88          91044720.0         86700210.0
     10          20167125.64          67471630.0         63483990.0
     11          19540073.37          61770140.0         57667040.0
     12          21730783.70          75252190.0         70714530.0
2017 1           21135104.97          75020810.0         69221860.0
     2           19268725.43          64818760.0         60120120.0
